In [70]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import norm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, auc, roc_curve, balanced_accuracy_score
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.decomposition import PCA
import os

In [71]:
path_to_directory="MDP/"
files = [arff for arff in os.listdir(path_to_directory) if arff.endswith(".arff")]

def toCsv(content): 
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent

In [72]:
# Main loop for reading and writing files
for zzzz,file in enumerate(files):
    with open(path_to_directory+file , "r") as inFile:
        content = inFile.readlines()
        name,ext = os.path.splitext(inFile.name)
        new = toCsv(content)
        with open(name+".csv", "w") as outFile:
            outFile.writelines(new)

In [73]:
cm1 = pd.read_csv('MDP/CM1.csv')
kc3 = pd.read_csv('MDP/KC3.csv')
#kc4 = pd.read_csv('MDP/KC4.csv')
mc2 = pd.read_csv('MDP/MC2.csv')
mw1 = pd.read_csv('MDP/MW1.csv')
pc1 = pd.read_csv('MDP/PC1.csv')
pc2 = pd.read_csv('MDP/PC2.csv')
pc3 = pd.read_csv('MDP/PC3.csv')
pc4 = pd.read_csv('MDP/PC4.csv')
jm1 = pd.read_csv('MDP/JM1.csv')
jm1.rename(columns={"label": "Defective"}, errors="raise", inplace=True)
kc1 = pd.read_csv('MDP/KC1.csv')
mc1 = pd.read_csv('MDP/MC1.csv')
pc5 = pd.read_csv('MDP/PC5.csv')

In [74]:
list_dataset = [kc1,pc2,mc2]
label_dataset = ['kc1']

list_test = [cm1,kc3,mc2,mw1,pc1,pc2,pc3,pc4,pc5,jm1,kc1,mc1] 
label_test = ['cm1','kc3','mc2','mw1','pc1','pc2','pc3','pc4','pc5','jm1','kc1','mc1']

my_index = range(0,len(list_test))
my_dataset = dict(zip(label_test,list_test))

df = list_dataset
df = pd.concat(df)
df

,LOC_BLANK,BRANCH_COUNT,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,LOC_EXECUTABLE,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,...,PARAMETER_COUNT,MAINTENANCE_SEVERITY,MODIFIED_CONDITION_COUNT,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUMBER_OF_LINES,PERCENT_COMMENTS,GLOBAL_DATA_COMPLEXITY,GLOBAL_DATA_DENSITY
0,6.0,15,1,3,8,8,1,45,23.87,27.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,17,0,7,9,8,1,82,69.72,22.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.0,31,0,0,16,13,10,95,59.66,33.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1,0,0,1,1,1,12,89.39,2.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1,0,0,1,1,1,3,16.84,1.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,9.0,23,5,9,12,4,8,41,42.25,28.27,...,3.0,0.67,11.0,19.0,32.0,0.18,68.0,25.45,5.0,0.42
120,1.0,3,0,0,2,1,1,9,10.82,9.75,...,2.0,0.50,1.0,2.0,6.0,0.17,12.0,0.00,2.0,1.00
121,8.0,3,5,8,2,1,1,43,69.69,27.79,...,0.0,0.50,1.0,2.0,21.0,0.03,66.0,23.21,2.0,1.00
122,9.0,11,0,33,6,2,1,23,19.50,40.24,...,2.0,0.17,5.0,10.0,24.0,0.09,68.0,58.93,5.0,0.83


In [75]:
def adjustdataset_list(list_df):
    dt_result = list_df.pop()
    
    for dt in list_df:
        if len(dt.columns) > len(dt_result.columns):
            dt_result = adjustdataset(dt,dt_result)
        else:
            dt_result = adjustdataset(dt_result,dt)    
    return dt_result

In [76]:
def adjustdataset(dt,dataframe):
    dt_copy = dt.copy()
    counter = 0
    
    for col in dataframe.columns:
        if not col in dt.columns:
            #print("Not Exist ",col)
            dt_copy.insert(counter,col,-1)
        counter += 1

    return pd.concat([dt_copy[dataframe.columns], dataframe])
    

In [77]:
def show_distribution(df, columns):
    
    counter = 0
    # Create a figure and a grid of subplots with 4 rows and 10 columns
    fig, axes = plt.subplots(5, 8, figsize=(20, 8))
    
    # Loop through each subplot and add content (optional)
    for i in range(5):
        for j in range(8):
            try:
                ax = axes[i, j]
                ax.set_title(columns[counter])
                ax.plot(df[columns[counter]].sort_values(), norm.pdf(df[columns[counter]].sort_values(), df[columns[counter]].mean(), df[columns[counter]].std())) 
                ax.set_xticks([])
                ax.set_yticks([])
                counter += 1
            except:
                pass

# Adjust layout to prevent overlapping titles/labels
plt.tight_layout()

# Display the plot
plt.show()

<Figure size 640x480 with 0 Axes>

In [78]:
def preprocessing(dataframe, imputer = None, pt = None, scaler = None, show = True):
    
    df = dataframe.copy()
    
    df.loc[(df['Defective'] == 'N'),'Defective']=0
    df.loc[(df['Defective'] == 'Y'),'Defective']=1
    
    change_dtypes = ['Defective']
    dict_dtypes = {d:'int64' for d in change_dtypes }
    df = df.astype(dict_dtypes)
    
    try:
        df.loc[(df['DECISION_DENSITY'] == '?'),'DECISION_DENSITY']= -1
        
        change_dtypes = ['DECISION_DENSITY']
        dict_dtypes = {d:'float64' for d in change_dtypes }
        df = df.astype(dict_dtypes)
    
    except:
        pass
        
    
    columns = df.columns
    
    if show:
        show_distribution(df,df.columns)
    
    if imputer :
        df = imputer.transform(df)
    else:
        imputer = KNNImputer(n_neighbors=2)
        df = imputer.fit_transform(df)
    
    df = pd.DataFrame(df)
    df.columns = columns
    
    if show:
        show_distribution(df,df.columns)
    
    y = df['Defective']
    X = df.drop('Defective',axis=1)
    
    if pt:
        X_norm = pt.transform(X)
    else:
        pt = PowerTransformer()
        X_norm = pt.fit_transform(X)
    
    X_norm = pd.DataFrame(X)
    X_norm.columns = X.columns
    
    if show:
        show_distribution(X_norm,X_norm.columns)

    if scaler:
        X_norm_scale = scaler.transform(X_norm)
    else:
        scaler = StandardScaler()
        X_norm_scale = scaler.fit_transform(X_norm)
        
    return pt, scaler, imputer, X_norm_scale, y



In [79]:
def heat_map(y,result,title):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    sns.set
    _, ax = plt.subplots(figsize=(10,10))
    ax = sns.heatmap(confusion_matrix(y, result), annot=True, fmt='d', cmap= ['#09acec','#089dd9','#078fc5','#0681b1','#05668d','#05368d'], annot_kws={"size": 40, "weight": "bold"})  
    labels = ['No Defect','Defect']
    ax.set_xticklabels(labels, fontsize=30);
    ax.set_yticklabels(labels, fontsize=30,rotation=0);
    ax.set_ylabel('Prediction', fontsize=30,rotation=0);
    ax.set_xlabel('Ground Truth', fontsize=30) #0,1
    ax.set_title(title)

In [80]:
def score(y,result):
    accuracy = accuracy_score(y, result)
    f1 = f1_score(y, result, average='weighted')
    precision = precision_score(y, result, average='weighted')
    recall = recall_score(y, result, average='weighted')
    print("Accuracy = " , accuracy, " Precision = ",precision, " Recall = ",recall, " F1-Score = ",f1 )

In [81]:
from sklearn.metrics import make_scorer

scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }
search_space={}

In [82]:
pt, scaler, imputer, X_norm_scale, y = preprocessing(df, show=False)
#X_norm_scale_smote, y_smote = X_norm_scale, y
smote = SMOTE(random_state=42)
X_norm_scale_smote, y_smote = smote.fit_resample(X_norm_scale, y)  

In [83]:
counter = 0
histories = []

lst_accu_stratified = []
lst_f1_stratified = []
lst_precision_stratified = []
lst_recall_stratified = []
lst_auc_stratified = []
lst_bauc_stratified = []
lst_gmean_stratified = []

lst_accu_stratified_val = []
lst_f1_stratified_val = []
lst_precision_stratified_val = []
lst_recall_stratified_val = []
lst_auc_stratified_val = []
lst_bauc_stratified_val = []
lst_gmean_stratified_val = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

for train_index, test_index in skf.split(X_norm_scale_smote, y_smote):

    x_train_fold, x_test_fold = X_norm_scale_smote[train_index], X_norm_scale_smote[test_index]
    y_train_fold, y_test_fold = y_smote[train_index], y_smote[test_index]
 
    rte = RandomTreesEmbedding(n_estimators=1000, random_state=0, max_depth=10).fit(x_train_fold)
    x_sparse_embedding_train = rte.transform(x_train_fold)
    x_sparse_embedding_test = rte.transform(x_test_fold)
    
    with open('rte-tree-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(rte, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    from sklearn.manifold import Isomap
    manifold = Isomap(n_components=100)
    x_sparse_manifold_train = manifold.fit_transform(x_sparse_embedding_train)
    x_sparse_manifold_test = manifold.transform(x_sparse_embedding_test)
    
    with open('manifold-tree-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(manifold, handle, protocol=pickle.HIGHEST_PROTOCOL)

    scaler2 = StandardScaler()
    x_sparse_manifold_train = scaler2.fit_transform(x_sparse_manifold_train)
    x_sparse_manifold_test = scaler2.transform(x_sparse_manifold_test)
    
    with open('scaler-tree-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(scaler2, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    input_X_train1 = layers.Input(
    shape=(x_sparse_manifold_train.shape[1],), name="X_train1")  
    
    input_X_train2 = layers.Input(shape=(x_train_fold.shape[1],), name="X_train2")

    layer_X_train1 = layers.Dense(64,activation='relu',name="X_train1_layer")(input_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.3)(layer_X_train1)
    
    layer_X_train2 = layers.Dense(64,activation='relu',name="X_train2_layer")(input_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.3)(layer_X_train2)
    
    # Merge all available features into a single large vector via concatenation
    concat = layers.concatenate([layer_X_train1, layer_X_train2])

    layer_final = layers.Dense(1, activation='sigmoid', name="classifier")(layer_X_train1)
    #sdp_classifier = Model(inputs=[input_X_train1, input_X_train2], outputs=layer_final)
    sdp_classifier = Model(inputs=input_X_train1, outputs=layer_final)


    checkpointer=ModelCheckpoint(filepath='sdp-tree-{0}.weights.best.keras'.format(counter),verbose=1,save_best_only=True)

    sdp_classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'] )
    
    hist = sdp_classifier.fit([x_sparse_manifold_train,x_train_fold], y_train_fold, 
                    batch_size=32, epochs=100, validation_split= 0.1, callbacks=[checkpointer], 
                    verbose=2, shuffle=True)
    
    with open('sdp-classifier-tree-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(sdp_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    sdp_classifier.load_weights('sdp-tree-{0}.weights.best.keras'.format(counter))

    histories.append(hist)

    results = sdp_classifier.predict([x_sparse_manifold_train,x_train_fold])
    results = np.round(results)
    
    lst_accu_stratified.append(accuracy_score(y_train_fold, results))
    lst_f1_stratified.append(f1_score(y_train_fold, results, average='weighted') )
    lst_precision_stratified.append(precision_score(results, y_train_fold, average='weighted') )
    lst_recall_stratified.append(recall_score(results, y_train_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_train_fold, results)
    lst_auc_stratified.append(auc(fpr, tpr))
    lst_bauc_stratified.append(balanced_accuracy_score(y_train_fold, results))
    
    results = sdp_classifier.predict([x_sparse_manifold_test,x_test_fold])
    results = np.round(results)
    
    lst_accu_stratified_val.append(accuracy_score(y_test_fold, results))
    lst_f1_stratified_val.append(f1_score(results, y_test_fold, average='weighted') )
    lst_precision_stratified_val.append(precision_score(results, y_test_fold, average='weighted') )
    lst_recall_stratified_val.append(recall_score(results, y_test_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_test_fold, results)
    lst_auc_stratified_val.append(auc(fpr, tpr))
    lst_bauc_stratified_val.append(balanced_accuracy_score(y_test_fold, results))
    
    print("Evaluate on training data")
    results = sdp_classifier.evaluate([x_sparse_manifold_train, x_train_fold], y_train_fold)
    print("test loss, test acc:", results)

    print("Evaluate on test data")
    results = sdp_classifier.evaluate([x_sparse_manifold_test , x_test_fold], y_test_fold)
    print("test loss, test acc:", results)
    counter += 1
    

dict_test = {  "train_accu":lst_accu_stratified, 
               "train_precision":lst_precision_stratified, "train_recall":lst_recall_stratified,
               "train_auc":lst_auc_stratified, "train_f1":lst_f1_stratified, "train_bauc":lst_bauc_stratified,
             
               "test_accu":lst_accu_stratified_val, 
               "test_precision":lst_precision_stratified_val, "test_recall":lst_recall_stratified_val, 
               "test_auc":lst_auc_stratified_val, "test_f1":lst_f1_stratified_val,"test_bauc":lst_bauc_stratified_val, 
               "Input": x_train_fold.shape[1], "RTE": x_sparse_manifold_train.shape[1]}

dict_test_result = pd.DataFrame.from_dict(dict_test)
dict_test_result.to_csv("hasil.csv")

with open('histories-tree-manifold.pickle'.format(counter), 'wb') as handle:
        pickle.dump(histories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100


ValueError: in user code:

    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/lib/python3.11/site-packages/keras/src/engine/input_spec.py", line 219, in assert_input_compatibility
        raise ValueEr

In [ ]:
dict_test_result

In [ ]:
best = np.argmax(dict_test_result['test_f1'])

dbfile = open('rte-tree-{0}.pickle'.format(best), 'rb')    
rte = pickle.load(dbfile)
dbfile.close()

dbfile = open('manifold-tree-{0}.pickle'.format(best), 'rb')    
manifold = pickle.load(dbfile)
dbfile.close()

dbfile = open('scaler-tree-{0}.pickle'.format(best), 'rb')    
scaler2 = pickle.load(dbfile)
dbfile.close()

dbfile = open('sdp-classifier-tree-{0}.pickle'.format(best), 'rb')    
sdp_classifier = pickle.load(dbfile)
dbfile.close()
sdp_classifier.load_weights('sdp-tree-{0}.weights.best.keras'.format(best))


In [ ]:
X_sparse_embedding = rte.transform(X_norm_scale)
X_sparse_manifold = manifold.transform(X_sparse_embedding)
X_sparse_manifold = scaler2.transform(X_sparse_manifold)
X_sparse_embedding.shape, X_sparse_manifold.shape

In [ ]:
print("Evaluate on training data")
results = sdp_classifier.evaluate([X_sparse_manifold, X_norm_scale], y, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
from sklearn.metrics import classification_report

results = sdp_classifier.predict([X_sparse_manifold,X_norm_scale])
results=np.round(results)
print(classification_report(y,results))

In [ ]:
list_result = []

for label in my_dataset:
    dt = adjustdataset(my_dataset[label], df)
    pt, scaler_dt, imputer, X_norm_scale_dt, y_dt = preprocessing(dt,imputer,pt, scaler, show=False)
    x_sparse_embedding_dt = rte.transform(X_norm_scale_dt)
    x_sparse_manifold_dt = manifold.transform(x_sparse_embedding_dt)
    x_sparse_manifold_dt = scaler2.transform(x_sparse_manifold_dt)
    results = sdp_classifier.predict([x_sparse_manifold_dt,X_norm_scale_dt])
    result  = np.round(results)
    fpr, tpr, thresholds = roc_curve(y_dt, result)
    list_result.append({'label':label, 'accuracy' : accuracy_score(y_dt, result),'precision':precision_score(y_dt, result, average='weighted'),
                  'recall': recall_score(y_dt, result, average='weighted'), 'auc':auc(fpr, tpr), 'gmean': balanced_accuracy_score(y_dt, result), 'bauc': balanced_accuracy_score(y_dt, result),
                        'f1-score':f1_score(y_dt, result, average='weighted')})
    #heat_map(y_dt,result,label)


    

In [ ]:
df_result = pd.DataFrame(list_result)
df_result

In [ ]:
df_result[['accuracy','precision','recall','auc','f1-score']].mean()

In [ ]:
sdp_classifier.summary()

In [ ]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    list_result.append({'Dataset':label, 'Instances':dt.shape[0], 'Features':dt.shape[1],
                        'Defective Instances': dt[dt['Defective'] == 'Y'].shape[0],
                        'Non-Defective Instances': dt[dt['Defective'] == 'N'].shape[0],})


    

In [ ]:
df_result = pd.DataFrame(list_result)
df_result

In [ ]:
import keras

dot_img_file = 'model_1.png'
keras.utils.plot_model(sdp_classifier, to_file=dot_img_file, show_shapes=True)

In [ ]:
sdp_classifier

In [ ]:
# Get the first dense layer by name
meta_model_1 = sdp_classifier.get_layer('X_train1_layer')
meta_model_2 = sdp_classifier.get_layer('X_train2_layer')
concatenate = sdp_classifier.get_layer('concatenate_3')
final_layer = sdp_classifier.get_layer('classifier')

# Retrieve the weights and biases of the dense layer
meta_model_1_weights, meta_model_1_biases = meta_model_1.get_weights()
meta_model_2_weights, meta_model_2_biases = meta_model_2.get_weights()
concatenate_weights = concatenate.get_weights()
final_layer_weights, final_layer_biases = final_layer.get_weights()

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(n_jobs=4, n_components=2, verbose = 1)
Y  = tsne.fit_transform(meta_model_1_weights)
tsne_ori = pd.DataFrame(Y, columns = ["tsne1", "tsne2"])

import plotly_express as px

figx = px.scatter(
    tsne_ori,
    x="tsne1",
    y="tsne2",
    #color="Defective",
    #hover_name="Defective",
    width=970,
    height=800,
    template="ggplot2",
    color_discrete_sequence= px.colors.qualitative.Alphabet,
    #facet_col="group_label",
    size_max=0.1,
)

figx.show()

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(n_jobs=4, n_components=2, verbose = 1)
Y  = tsne.fit_transform(meta_model_2_weights)
tsne_ori = pd.DataFrame(Y, columns = ["tsne1", "tsne2"])

import plotly_express as px

figx = px.scatter(
    tsne_ori,
    x="tsne1",
    y="tsne2",
    #color="Defective",
    #hover_name="Defective",
    width=970,
    height=800,
    template="ggplot2",
    color_discrete_sequence= px.colors.qualitative.Alphabet,
    #facet_col="group_label",
    size_max=0.1,
)

figx.show()

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(n_jobs=4, n_components=2, verbose = 1)
Y  = tsne.fit_transform(final_layer_weights)
tsne_ori = pd.DataFrame(Y, columns = ["tsne1", "tsne2"])

import plotly_express as px

figx = px.scatter(
    tsne_ori,
    x="tsne1",
    y="tsne2",
    #color="Defective",
    #hover_name="Defective",
    width=970,
    height=800,
    template="ggplot2",
    color_discrete_sequence= px.colors.qualitative.Alphabet,
    #facet_col="group_label",
    size_max=0.1,
)

figx.show()